In [2]:
#r "nuget:Deedle.Interactive,3.0.0-beta.1"

Installed Packages Deedle.Interactive, 3.0.0-beta.1

Loading extensions from `Deedle.Interactive.dll`

In [3]:
open Deedle

In [4]:
let customers = Frame.ReadCsv("customers.csv")
let transactions = Frame.ReadCsv("transactions.csv")

In [5]:
customers
|> Frame.take 5

0,->,1001,Johnny Awesome,3,ON,1,wealth
1,->,1002,Bob Marley,4,ON,1,personal_banking
2,->,1004,Taylor Swift,2,AB,2,hnw
3,->,1101,James Dean,3,BC,3,wealth
4,->,1006,Stephen Smith,5,ON,4,hnw


In [6]:
transactions
|> Frame.take 5

0,->,1,1001,Debit,165.78
1,->,2,1001,Credit,42.1
2,->,4,1001,Credit,103.03
3,->,5,1001,Credit,56.6
4,->,6,1002,Debit,214.34


In [7]:
customers.RenameColumn("id", "customer_id")

In [8]:
let customerIndexed = customers.IndexRows<int>("customer_id")

In [9]:
customerIndexed

1001,->,Johnny Awesome,3,ON,1,wealth
1002,->,Bob Marley,4,ON,1,personal_banking
1004,->,Taylor Swift,2,AB,2,hnw
1101,->,James Dean,3,BC,3,wealth
1006,->,Stephen Smith,5,ON,4,hnw
:,,...,...,...,...,...
1354,->,Jessica Fast,4,BC,5,personal_banking
1005,->,Mariah Anita Smith,2,BC,3,personal_banking
1008,->,Robert Optimus,3,MB,10,hnw
1585,->,Steven Prime,5,AB,2,wealth
1475,->,Stephanie Element,2,ON,5,personal_banking


In [10]:
transactions

0,->,1,1001,Debit,165.78
1,->,2,1001,Credit,42.1
2,->,4,1001,Credit,103.03
3,->,5,1001,Credit,56.6
4,->,6,1002,Debit,214.34
:,,...,...,...,...
29,->,31,1354,Credit,438.29
30,->,32,1005,Debit,122.06
31,->,33,1008,Cash,166.44
32,->,34,1585,Debit,307.91
33,->,35,1475,Cash,266.30


In [11]:
transactions.RenameColumn("id", "txn_id")

In [12]:
transactions

0,->,1,1001,Debit,165.78
1,->,2,1001,Credit,42.1
2,->,4,1001,Credit,103.03
3,->,5,1001,Credit,56.6
4,->,6,1002,Debit,214.34
:,,...,...,...,...
29,->,31,1354,Credit,438.29
30,->,32,1005,Debit,122.06
31,->,33,1008,Cash,166.44
32,->,34,1585,Debit,307.91
33,->,35,1475,Cash,266.30


Deedle doesn't support indexing columns with duplicate values. How do we join 2 dataframes?

In [13]:
// Series.zip workOrders workOrderScores
// |> Series.map(fun _ (orders, scores) ->
//     match orders, scores with
//     | OptionalValue.Present s1, OptionalValue.Present s2 ->
//         // There is a frame with some rows with the specified code in both
//         // work orders and work order scores - we return a cross product of their rows
//         [ for r1 in s1.Rows.Values do
//           for r2 in s2.Rows.Values do
//           // Drop workOrderCode from one series (they are the same in both)
//           // and append the two rows & return that as the result
//           yield (r1, (Series.filter (fun k _ -> k <> "workOrderCode") r2)) ]
//         // |> Frame.ofRowsOrdinal
//     // If left or right value is missing, we just return the columns
//     // that are available (others will be filled with NaN)
//     | OptionalValue.Present s, s2
//     | s1, OptionalValue.Present s2 -> failwith "blah"
//     | _ -> s
//   )
// // |> Frame.unnest
// // |> Frame.indexRowsOrdinally

In [14]:
// transactions.SortColumnsByKey().Join(customers)

Write a function to display a Series object with only the names of Wealth customers.

In [15]:
customers
|> Frame.filterRowValues (fun row -> row.GetAs<string>("cust_category") = "wealth")
|> Frame.indexRowsString "cust_name"
|> Frame.sliceCols ["cust_category"]
// |> (fun df -> df.GetColumn<string>("cust_category"))

Johnny Awesome,->,wealth
James Dean,->,wealth
Maria Alva,->,wealth
Steven Prime,->,wealth


Write a function to identify all PersonalBanking customers with more than 3 products.

In [16]:
// customers[(customers['cust_category'] == 'personal_banking') & (customers['num_products'] > 3)]

customers
|> Frame.filterRowValues (
    fun row ->
        row.GetAs<string>("cust_category") = "personal_banking"
        && row?num_products > 3
    )


1,->,1002,Bob Marley,4,ON,1,personal_banking
8,->,1354,Jessica Fast,4,BC,5,personal_banking
